In [1]:
import numpy as np
import pandas as pd

import json
import keras
import tensorflow
import tensorflow_addons as tfa
from keras import layers, Model
from keras.models import Sequential
from keras.applications import DenseNet201
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

Using TensorFlow backend.


In [2]:
csv_path = './UrbanSound8K/metadata/UrbanSound8K.csv'
spec_path = "./numpySpectrograms/"
test_size = 0.2
val_size = 0.2
batch_size = 16
num_classes = 5
target_class = 4
model_save_path = "ovr_basemodel" + str(target_class)

In [3]:
class SpecLoader(keras.utils.Sequence):
  def __init__(self, x_set, y_set, batch_size, spec_dir):
    self.x, self.y = x_set, y_set
    self.batch_size = batch_size
    self.spec_dir = spec_dir

  def __len__(self):
    return int(np.ceil(len(self.x) / self.batch_size))

  def __getitem__(self, idx):
    batch_x = self.x[idx * self.batch_size:(idx + 1) *
    self.batch_size]
    batch_y = self.y[idx * self.batch_size:(idx + 1) *
        self.batch_size]

    batchSpecs = []
    for fileName in batch_x:
        spec = np.load(self.spec_dir + fileName + ".npy")
        batchSpecs.append(spec.transpose())
    return np.array(batchSpecs), np.array(batch_y)

In [4]:
data_df = pd.read_csv(csv_path)
data_df_known = data_df.loc[data_df["classID"] < num_classes]

X_trainval, X_test, y_trainval, y_test = train_test_split(data_df_known['slice_file_name'].tolist(), data_df_known['classID'].tolist(), test_size=test_size, random_state = 42)

X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=val_size, random_state = 42)

y_train = [1 if j == target_class else 0 for j in y_train]
y_val = [1 if j == target_class else 0 for j in y_val]
y_test = [1 if j == target_class else 0 for j in y_test]

train_loader = SpecLoader(X_train, y_train, batch_size, spec_path)
val_loader = SpecLoader(X_val, y_val, batch_size, spec_path)
test_loader = SpecLoader(X_test, y_test, batch_size, spec_path)

In [5]:
densenet = DenseNet201(
            include_top=False,
            weights="imagenet",
            input_tensor=None,
            input_shape=None,
            pooling="avg")


model = Sequential()
model.add(densenet)
model.add(layers.Dense(2, activation="softmax"))
model.summary()
model.compile(
      optimizer="Adam",
      loss="sparse_categorical_crossentropy",
      metrics=["accuracy"])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet201 (Model)          (None, 1920)              18321984  
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 3842      
Total params: 18,325,826
Trainable params: 18,096,770
Non-trainable params: 229,056
_________________________________________________________________


In [6]:
earlystopping = EarlyStopping(
                    patience=5, 
                    restore_best_weights=True)
checkpoint = ModelCheckpoint(
                    model_save_path, 
                    monitor="val_accuracy", 
                    save_best_only=True)

model.fit(x=train_loader,
          validation_data=val_loader,
          callbacks=[checkpoint, earlystopping],
          epochs=70,
          verbose=1
         )

Epoch 1/70
178/178 [==============================] - 192s 1s/step - loss: 0.3259 - accuracy: 0.8913 - val_loss: 3.8005 - val_accuracy: 0.6474
Epoch 2/70
178/178 [==============================] - 53s 297ms/step - loss: 0.1847 - accuracy: 0.9273 - val_loss: 0.2555 - val_accuracy: 0.8815
Epoch 3/70
178/178 [==============================] - 53s 296ms/step - loss: 0.1597 - accuracy: 0.9439 - val_loss: 0.0975 - val_accuracy: 0.9069
Epoch 4/70
178/178 [==============================] - 53s 296ms/step - loss: 0.1128 - accuracy: 0.9577 - val_loss: 0.3626 - val_accuracy: 0.9506
Epoch 5/70
178/178 [==============================] - 52s 295ms/step - loss: 0.0933 - accuracy: 0.9647 - val_loss: 0.0140 - val_accuracy: 0.9549
Epoch 6/70
178/178 [==============================] - 52s 295ms/step - loss: 0.0699 - accuracy: 0.9728 - val_loss: 0.1126 - val_accuracy: 0.9281
Epoch 7/70
178/178 [==============================] - 53s 298ms/step - loss: 0.1079 - accuracy: 0.9615 - val_loss: 0.0790 - val_accu

In [7]:
model.load_weights(model_save_path)
y_predicted = model.predict_classes(x=test_loader, batch_size=None)

In [8]:
def evaluate(predicted, expected):
    acc = np.mean(np.array(predicted) == np.array(expected))
    print("Overall accuracy: {}".format(acc))
    acc_dict = {}
    for i in range(len(expected)):
        expected_class = expected[i]
        if expected_class not in acc_dict:
            acc_dict[expected_class] = [0, 0]
        acc_dict[expected_class][1] += 1
        if expected_class == predicted[i]:
            acc_dict[expected_class][0] += 1
    for k,v in acc_dict.items():
        print("Accuracy for class {}: {}".format(k, v[0]/v[1]))

print(model.evaluate(x=train_loader))
print(model.evaluate(x=val_loader))
print(model.evaluate(x=test_loader))
evaluate(y_predicted, y_test)

178/178 [==============================] - 12s 65ms/step
[0.0009712110040709376, 0.9657727479934692]
45/45 [==============================] - 3s 65ms/step
[0.01401529274880886, 0.9548659920692444]
56/56 [==============================] - 4s 65ms/step
[0.061140093952417374, 0.9525959491729736]
Overall accuracy: 0.9525959367945824
Accuracy for class 0: 0.9551820728291317
Accuracy for class 1: 0.9418604651162791
